In [36]:
#####################
# IMPORT LIBS
#####################

import pandas as pd
import numpy as np
from pathlib import Path
import wandb
import datetime
import os
import random
import joblib
import shutil

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from scoring import local_scorer
import scipy
from sklearn.metrics import f1_score, precision_score, recall_score

from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostRegressor, CatBoostClassifier
from xgboost import XGBClassifier


#####################
# SET CONSTANTS
#####################

INPUT_PATH = Path('../input')
OUTPUT_PATH = Path('../output')
TRAIN_PATH = INPUT_PATH 

TARGET_COLUMNS = ['sale_flg', 'sale_amount', 'contacts']
FIXED_SEEDS = [948, 534, 432, 597, 103, 21, 2242, 17, 20, 29]

RANDOM_SEED = 4444
USE_WANDB = False
CURRENT_TIME = str(datetime.datetime.now()).replace(' ', '_').split('.')[0]

def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(RANDOM_SEED)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

In [2]:
###############
# Config
###############

n_seed = 5
n_fold = 3
retrain_after_valid = True
make_submission = False

In [3]:
if USE_WANDB:
    wandb.login()
    run = wandb.init(project="idao-2021-finals", name = f'{CURRENT_TIME}') # todo add config here

In [4]:
# # %%time

# transactions = pd.read_csv(INPUT_PATH / 'trxn.csv')
# assets_under_management = pd.read_csv(INPUT_PATH / 'aum.csv')
# balance = pd.read_csv(INPUT_PATH / 'balance.csv')
# client = pd.read_csv(INPUT_PATH / 'client.csv')
# campaigns = pd.read_csv(INPUT_PATH / 'com.csv')
# deals = pd.read_csv(INPUT_PATH / 'deals.csv')
# dict_merchant_category_code = pd.read_csv(INPUT_PATH / 'dict_mcc.csv')
# payments = pd.read_csv(INPUT_PATH / 'payments.csv')
# funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
# appl = pd.read_csv(INPUT_PATH / 'appl.csv')

In [5]:
def get_feature_total(df, col_name):
    return data['client_id'].map(df.groupby(['client_id', col_name]).size().index.get_level_values('client_id').value_counts()).fillna(0)


def get_feature_most_common(df, col_name, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name].agg(lambda x: scipy.stats.mode(x)[0][0])).fillna(fill_na_value)


def get_feature_max(df, col_name, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name].max()).fillna(fill_na_value)


def get_feature_min(df, col_name, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name].min()).fillna(fill_na_value)


def get_feature_mean(df, col_name, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name].mean()).fillna(fill_na_value)


def get_feature_std(df, col_name, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name].std()).fillna(fill_na_value)


def get_feature_max_min(df, col_name, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name].agg(lambda x: x.max() - x.min())).fillna(fill_na_value)


def get_feature_timedelta(df, col_name):
    return data['client_id'].map(df.groupby('client_id')[col_name].agg(lambda x: (x.max() - x.min()).days)).fillna(-1)


def get_feature_diff(df, col_name1, col_name2, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name1].sum() - df.groupby('client_id')[col_name2].sum()).fillna(fill_na_value)


def get_feature_rate(df, col_name1, col_name2, fill_na_value):
    return data['client_id'].map(df.groupby('client_id')[col_name1].sum() / (df.groupby('client_id')[col_name2].sum() + 1e-12)).fillna(fill_na_value)


def create_features_transactions(data):
    
    transactions = pd.read_csv(INPUT_PATH / 'trxn.csv')
    dict_merchant_category_code = pd.read_csv(INPUT_PATH / 'dict_mcc.csv')
    
    transactions['mcc_cd'] = transactions['mcc_cd'].fillna(-2)
    transactions['txn_city'] = transactions['txn_city'].fillna('<UNK>')
    transactions['tsp_name'] = transactions['tsp_name'].fillna('<UNK>')
    transactions['txn_comment_2'] = transactions['txn_comment_2'].fillna('<UNK>')

    transactions = transactions.merge(dict_merchant_category_code, on='mcc_cd', how='left')
    del dict_merchant_category_code
    transactions['brs_mcc_group'] = transactions['brs_mcc_group'].fillna('<UNK>')
    transactions['brs_mcc_subgroup'] = transactions['brs_mcc_subgroup'].fillna('<UNK>')
    
    data['total_transactions'] = data['client_id'].map(transactions.groupby('client_id').size()).fillna(0)
#     data['total_transactions_cards'] = get_feature_total(transactions, 'card_id')

#     data['total_transaction_amount'] = data['client_id'].map(transactions.groupby('client_id')['tran_amt_rur'].sum()).fillna(0) # add monthly, daily, etc
    data['mean_transaction_amt'] = get_feature_mean(transactions, 'tran_amt_rur', -1) # add monthly, daily, etc
    data['std_transaction_amount'] = get_feature_std(transactions, 'tran_amt_rur', -1) # add monthly, daily, etc
    
    data['total_transactions_mcc_cd'] = get_feature_total(transactions, 'mcc_cd')
    data['total_transactions_share_mcc_cd'] = (data['total_transactions_mcc_cd'] / data['total_transactions']).fillna(0)
#     data['most_common_transactions_mcc_cd'] = get_feature_most_common(transactions, 'mcc_cd', -1)
    
    data['total_transactions_merchant_cd'] = get_feature_total(transactions, 'merchant_cd')
    data['total_share_transactions_merchant_cd'] = (data['total_transactions_merchant_cd'] / data['total_transactions']).fillna(0)
#     data['most_common_transactions_merchant_cd'] = get_feature_most_common(transactions, 'merchant_cd', -1)
    
    data['total_transactions_txn_city'] = get_feature_total(transactions, 'txn_city')
    data['total_share_transactions_txn_city'] = (data['total_transactions_txn_city'] / data['total_transactions']).fillna(0)
    data['most_common_transactions_txn_city'] = get_feature_most_common(transactions, 'txn_city', '<unknown>')
    
    data['total_transactions_tsp_name'] = get_feature_total(transactions, 'tsp_name')
    data['total_share_transactions_tsp_name'] = (data['total_transactions_tsp_name'] / data['total_transactions']).fillna(0)
    data['most_common_transactions_tsp_name'] = get_feature_most_common(transactions, 'tsp_name', '<unknown>')
    
#     data['total_transactions_txn_comment_1'] = get_feature_total(transactions, 'txn_comment_1')
#     data['most_common_transactions_txn_comment_1'] = get_feature_most_common(transactions, 'txn_comment_1', '<unknown>')
    
#     data['total_transactions_txn_comment_2'] = get_feature_total(transactions, 'txn_comment_2')
#     data['most_common_transactions_txn_comment_2'] = get_feature_most_common(transactions, 'txn_comment_2', '<unknown>')
    
#     data['total_transactions_brs_mcc_group'] = get_feature_total(transactions, 'brs_mcc_group')
#     data['most_common_transactions_brs_mcc_group'] = get_feature_most_common(transactions, 'brs_mcc_group', '<unknown>')
    
#     data['total_transactions_brs_mcc_subgroup'] = get_feature_total(transactions, 'brs_mcc_subgroup')
#     data['most_common_transactions_brs_mcc_subgroup'] = get_feature_most_common(transactions, 'brs_mcc_subgroup', '<unknown>')
    
    del transactions

    return data


def create_features_aum(data):
    
    assets_under_management = pd.read_csv(INPUT_PATH / 'aum.csv')
    
    data['total_aum'] = data['client_id'].map(assets_under_management.groupby('client_id').size()).fillna(0)
    
#     data['total_aum_product_code'] = get_feature_total(assets_under_management, 'product_code')
#     data['most_common_aum_product_code'] = get_feature_most_common(assets_under_management, 'product_code', '<unknown>').value_counts()
    
    data['mean_aum_balance_rur_amt'] = get_feature_mean(assets_under_management, 'balance_rur_amt', -1)
    data['std_aum_balance_rur_amt'] = get_feature_std(assets_under_management, 'balance_rur_amt', -1)
    data['max_min_aum_balance_rur_amt'] = get_feature_max_min(assets_under_management, 'balance_rur_amt', -1)
    
    del assets_under_management
    
    return data


def create_features_balance(data):
    
    balance = pd.read_csv(INPUT_PATH / 'balance.csv')
    
    balance['crncy_cd'] = balance['crncy_cd'].fillna(-2)
    balance['prod_cat_name'] = balance['prod_cat_name'].fillna('<UNK>')
    balance['prod_group_name'] = balance['prod_group_name'].fillna('<UNK>')
    
    data['total_balance'] = data['client_id'].map(balance.groupby('client_id').size()).fillna(0)
    
#     data['total_balance_crncy_cd'] = get_feature_total(balance, 'crncy_cd')
#     data['most_common_balance_crncy_cd'] = get_feature_most_common(balance, 'crncy_cd', -1)
    
#     data['total_balance_eop_bal_sum_rur'] = get_feature_total(balance, 'eop_bal_sum_rur')
    data['total_share_balance_eop_bal_sum_rur'] = (get_feature_total(balance, 'eop_bal_sum_rur') / data['total_balance']).fillna(0)
    data['mean_balance_eop_bal_sum_rur'] = get_feature_mean(balance, 'eop_bal_sum_rur', -9999)
    data['std_balance_eop_bal_sum_rur'] = get_feature_std(balance, 'eop_bal_sum_rur', -9999)
    
    data['total_balance_min_bal_sum_rur'] = get_feature_total(balance, 'min_bal_sum_rur')
    data['total_share_balance_min_bal_sum_rur'] = (data['total_balance_min_bal_sum_rur'] / data['total_balance']).fillna(0)
    data['mean_balance_min_bal_sum_rur'] = get_feature_mean(balance, 'min_bal_sum_rur', -9999)
    data['std_balance_min_bal_sum_rur'] = get_feature_std(balance, 'min_bal_sum_rur', -9999)
    
    data['total_balance_max_bal_sum_rur'] = get_feature_total(balance, 'max_bal_sum_rur')
    data['total_share_balance_max_bal_sum_rur'] = (data['total_balance_max_bal_sum_rur'] / data['total_balance']).fillna(0)
    data['mean_balance_max_bal_sum_rur'] = get_feature_mean(balance, 'max_bal_sum_rur', -9999)
    data['std_balance_max_bal_sum_rur'] = get_feature_std(balance, 'max_bal_sum_rur', -9999)
    
    data['total_balance_avg_bal_sum_rur'] = get_feature_total(balance, 'avg_bal_sum_rur')
    data['total_share_balance_avg_bal_sum_rur'] = (data['total_balance_avg_bal_sum_rur'] / data['total_balance']).fillna(0)
    data['mean_balance_avg_bal_sum_rur'] = get_feature_mean(balance, 'avg_bal_sum_rur', -9999)
#     data['std_balance_avg_bal_sum_rur'] = get_feature_std(balance, 'avg_bal_sum_rur', -9999)
    data['max_min_balance_avg_bal_sum_rur'] = get_feature_max_min(balance, 'avg_bal_sum_rur', -9999)
    
#     data['total_balance_prod_cat_name'] = get_feature_total(balance, 'prod_cat_name')
#     data['most_common_balance_prod_cat_name'] = get_feature_most_common(balance, 'prod_cat_name', '<unknown>')
    
    data['total_balance_prod_group_name'] = get_feature_total(balance, 'prod_group_name')
#     data['most_common_balance_prod_group_name'] = get_feature_most_common(balance, 'prod_group_name', '<unknown>')
    
    del balance

    return data


def create_features_client(data):
    
    client = pd.read_csv(INPUT_PATH / 'client.csv')
    
    client = client.rename(columns={
        'gender': 'client_gender',
        'age': 'client_age',
        'region': 'client_region',
        'city': 'client_city',
        'citizenship': 'client_citizenship',
        'education': 'client_education',
        'job_type': 'client_job_type'
    })
    
    data = data.merge(client, on='client_id')
#     data['match_client_region-region_cd'] = (data['client_region'] == data['region_cd']).astype(int)
    data = data.drop(['client_citizenship', 'client_job_type', 'client_gender'], axis=1)
    
    del client
    
    return data


def create_features_campaigns(data):

    campaigns = pd.read_csv(INPUT_PATH / 'com.csv')
    
    campaigns['prod'] = campaigns['prod'].fillna('<UNK>')
    
    data['total_campaigns'] = data['client_id'].map(campaigns.groupby('client_id').size()).fillna(0)
    
#     data['total_campaigns_agr_flg'] = get_feature_total(campaigns, 'agr_flg')
    data['mean_campaigns_agr_flg'] = get_feature_mean(campaigns, 'agr_flg', -1)
    
#     data['total_campaigns_otkaz'] = get_feature_total(campaigns, 'otkaz')
    data['mean_campaigns_otkaz'] = get_feature_mean(campaigns, 'otkaz', -1)
    
#     data['total_campaigns_dumaet'] = get_feature_total(campaigns, 'dumaet')
    data['mean_campaigns_dumaet'] = get_feature_mean(campaigns, 'dumaet', -1)
    
#     data['total_campaigns_ring_up_flg'] = get_feature_total(campaigns, 'ring_up_flg')
#     data['most_common_campaigns_ring_up_flg'] = get_feature_most_common(campaigns, 'ring_up_flg', -1)
    
#     data['total_campaigns_count_comm'] = get_feature_total(campaigns, 'count_comm')
#     data['most_common_campaigns_count_comm'] = get_feature_most_common(campaigns, 'count_comm', -1)
    
#     data['total_campaigns_channel'] = get_feature_total(campaigns, 'channel')
#     data['most_common_campaigns_channel'] = get_feature_most_common(campaigns, 'channel', '<unknown>')
    
#     data['total_campaigns_prod'] = get_feature_total(campaigns, 'prod')
    data['most_common_campaigns_prod'] = get_feature_most_common(campaigns, 'prod', '<unknown>')
    
#     data['diff_campaigns_otkaz-agr_flg'] = get_feature_diff(campaigns, 'otkaz', 'agr_flg', -999)
    
    data['rate_campaigns_otkaz-count_comm'] = get_feature_rate(campaigns, 'otkaz', 'count_comm', -999)
    data['rate_campaigns_agr_flg-count_comm'] = get_feature_rate(campaigns, 'agr_flg', 'count_comm', -999)
    data['rate_campaigns_not_ring_up_flg-count_comm'] = get_feature_rate(campaigns, 'not_ring_up_flg', 'count_comm', -999)
    data['rate_campaigns_ring_up_flg-count_comm'] = get_feature_rate(campaigns, 'ring_up_flg', 'count_comm', -999)
    
    del campaigns
    
    return data


def create_features_deals(data):
    
    deals = pd.read_csv(INPUT_PATH / 'deals.csv')
    
    deals['crncy_cd'] = deals['crncy_cd'].fillna(-2)
    deals['agrmnt_rate_active'] = deals['agrmnt_rate_active'].fillna(-2)
    deals['agrmnt_rate_passive'] = deals['agrmnt_rate_passive'].fillna(-2)
    deals['agrmnt_sum_rur'] = deals['agrmnt_sum_rur'].fillna(-2)
    deals['prod_type_name'] = deals['prod_type_name'].fillna('<UNK>')
    deals['argmnt_close_start_days'] = (pd.to_datetime(deals['agrmnt_close_dt']) - pd.to_datetime(deals['agrmnt_start_dt'])).dt.days.fillna(-2)
    
    data['total_deals'] = data['client_id'].map(deals.groupby('client_id').size()).fillna(0)
    
#     data['total_deals_crncy_cd'] = get_feature_total(deals, 'crncy_cd')
#     data['most_common_deals_crncy_cd'] = get_feature_most_common(deals, 'crncy_cd', -1)
    
    data['total_deals_agrmnt_rate_active'] = get_feature_total(deals, 'agrmnt_rate_active')
    data['max_deals_agrmnt_rate_active'] = get_feature_max(deals, 'agrmnt_rate_active', -1)
    
#     data['total_deals_agrmnt_rate_passive'] = get_feature_total(deals, 'agrmnt_rate_passive')
    data['max_deals_agrmnt_rate_passive'] = get_feature_max(deals, 'agrmnt_rate_passive', -1)
    
    data['total_deals_agrmnt_sum_rur'] = get_feature_total(deals, 'agrmnt_sum_rur')
    data['mean_deals_agrmnt_sum_rur'] = get_feature_mean(deals, 'agrmnt_sum_rur', -1)
    data['std_deals_agrmnt_sum_rur'] = get_feature_std(deals, 'agrmnt_sum_rur', -1)
    
    data['total_deals_prod_type_name'] = get_feature_total(deals, 'prod_type_name')
    data['most_common_deals_prod_type_name'] = get_feature_most_common(deals, 'prod_type_name', '<unknown>')
    
    data['total_deals_argmnt_close_start_days'] = get_feature_total(deals, 'argmnt_close_start_days')
    data['max_deals_argmnt_close_start_days'] = get_feature_max(deals, 'argmnt_close_start_days', -1)
#     data['min_deals_argmnt_close_start_days'] = get_feature_min(deals, 'argmnt_close_start_days', -1)
    data['mean_deals_argmnt_close_start_days'] = get_feature_mean(deals, 'argmnt_close_start_days', -1)
    data['std_deals_argmnt_close_start_days'] = get_feature_std(deals, 'argmnt_close_start_days', -1)
    
    del deals
    
    return data


def create_features_payments(data):
    
    payments = pd.read_csv(INPUT_PATH / 'payments.csv')
    
    payments['day_dt'] = pd.to_datetime(payments['day_dt'])
    
    data['total_payments'] = data['client_id'].map(payments.groupby('client_id').size()).fillna(0)
    
    data['mean_payments_sum_rur'] = get_feature_mean(payments, 'sum_rur', -1)
    data['std_payments_sum_rur'] = get_feature_std(payments, 'sum_rur', -1)
    data['min_payments_sum_rur'] = get_feature_min(payments, 'sum_rur', -1)
    data['max_payments_sum_rur'] = get_feature_max(payments, 'sum_rur', -1)
    
#     data['total_payments_pmnts_name'] = get_feature_total(payments, 'pmnts_name')
#     data['most_common_payments_pmnts_name'] = get_feature_most_common(payments, 'pmnts_name', '<unknown>')
    
    # payments 
#     data['last_known_salary'] = data['client_id'].map(payments.groupby('client_id').apply(lambda x: x['sum_rur'].iloc[0])).fillna(-1)
#     data['total_recieved_salary'] = data['client_id'].map(payments.groupby('client_id').apply(lambda x: x['sum_rur'].sum())).fillna(-1)
    
    data['timedelta_payments_day_dt'] = get_feature_timedelta(payments, 'day_dt')
    
    del payments
    
    return data


def create_features_appl(data):
    
    appl = pd.read_csv(INPUT_PATH / 'appl.csv')
    
    appl['appl_stts_name_dc'] = appl['appl_stts_name_dc'].fillna('<UNK>')
    appl['appl_sale_channel_name'] = appl['appl_sale_channel_name'].fillna('<UNK>')
    appl['month_end_dt'] = pd.to_datetime(appl['month_end_dt'])
    
    data['total_appl'] = data['client_id'].map(appl.groupby('client_id').size()).fillna(0)
    
#     data['total_appl_prod_group_name'] = get_feature_total(appl, 'appl_prod_group_name')
#     data['most_common_appl_prod_group_name'] = get_feature_most_common(appl, 'appl_prod_group_name', '<unknown>')
    
    data['total_appl_prod_type_name'] = get_feature_total(appl, 'appl_prod_type_name')
    data['most_common_appl_prod_type_name'] = get_feature_most_common(appl, 'appl_prod_type_name', '<unknown>')
    
#     data['total_appl_stts_name_dc'] = get_feature_total(appl, 'appl_stts_name_dc')
#     data['most_common_appl_stts_name_dc'] = get_feature_most_common(appl, 'appl_stts_name_dc', '<unknown>')
    
#     data['total_appl_sale_channel_name'] = get_feature_total(appl, 'appl_sale_channel_name')
#     data['most_common_appl_sale_channel_name'] = get_feature_most_common(appl, 'appl_sale_channel_name', '<unknown>')
    
    data['timedelta_appl_month_end_dt'] = get_feature_timedelta(appl, 'month_end_dt')
    
    del appl
    
    return data


def create_features_funnel(data):
    
    return data

In [6]:
%%time
# create features

funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')

data = funnel.copy()

del funnel

data = create_features_transactions(data)
data = create_features_aum(data)
data = create_features_balance(data)
data = create_features_client(data)
data = create_features_campaigns(data)
data = create_features_deals(data)
data = create_features_payments(data)
data = create_features_appl(data)
data = create_features_funnel(data)

/opt/conda/lib/python3.7/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


CPU times: user 32.5 s, sys: 721 ms, total: 33.2 s
Wall time: 33.2 s


In [7]:
# fill_cols = ['gender', 'citizenship', 'education', 'job_type', 'most_common_txn_city', 'most_common_tsp_name', 'most_common_txn_comment_1', 'most_common_txn_comment_2']# 'most_common_txn_city', 'most_common_tsp_name', 'most_common_txn_comment_1', 'most_common_txn_comment_2']
for c in data.columns:
    col_type = data[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        data[c] = data[c].astype('str')

In [8]:
X = data.drop(columns = TARGET_COLUMNS + ['client_id'])
Y = data[TARGET_COLUMNS[0]]

## Train

In [9]:
import shutil
try:
    os.mkdir(OUTPUT_PATH / 'models')
except:
    shutil.rmtree(OUTPUT_PATH / 'models')
    os.mkdir(OUTPUT_PATH / 'models')

In [10]:
cat_feats =[]
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        X[c] = X[c].astype(str)
        cat_feats.append(np.argwhere(X.columns == c)[0][0])

## Hyperparameter tuning

In [11]:
# from quick_hyperopt import quick_hyperopt
# cb_params = quick_hyperopt(X, Y, 'cb', 10, cat_features = cat_feats)

## Training

In [12]:
def running_train(X_train, Y_train, X_val, Y_val, i_fold=None, seed=None, params = None):
    # prepare for train
    
    params = {
              "thread_count":-1,
              "random_seed": seed,
#               "one_hot_max_size": 5
#               "learning_rate": 0.01,
#               "boosting_type":"Plain",
#               "leaf_estimation_iterations":1,
#               "iterations": 200,
#               "random_strength": 2,
#               "depth": 7,
#               "l2_leaf_reg": 1,
#               "eval_metric": 'F1',
#               'verbose': 10
#         'objective': 'quantile',
              }
    
    
    #model = LGBMRegressor(**params) # define model here
    model = CatBoostClassifier(**params)
    
    
    # Fit and save model
    
    if X_val is None:
        model.fit(X_train, Y_train, verbose=False, cat_features=cat_feats)
    else:
        model.fit(X_train, Y_train,   eval_set=(X_val, Y_val), early_stopping_rounds=500, verbose=False, cat_features=cat_feats)
    #joblib.dump(model, OUTPUT_PATH / 'models' / f'catboost_{i_fold}_{seed}_{CURRENT_TIME}.pkl')
    model.save_model(OUTPUT_PATH / 'models' / f'catboost_{i_fold}_{seed}_{CURRENT_TIME}')

In [ ]:
%%time

oof = np.zeros((X.shape[0], n_seed)) # cv_score
seeds = []
for i_seed in range(n_seed):
    seed = FIXED_SEEDS[i_seed]
    seed_everything(seed)

    seeds.append(seed)
    print('Seed: {}, {}/{}'.format(seed, i_seed + 1, n_seed))
    
    if n_fold != 1:
        kf = KFold(n_splits=n_fold, random_state=seed, shuffle=True)
        split_indexes = kf.split(X, Y)
    else:
        split_indexes = [train_test_split(np.arange(X.shape[0]), random_state=seed, shuffle = True)]
    
    for i_fold, (train_idx, val_idx) in enumerate(split_indexes):
#         print("# Fold: {}/{} (seed: {}/{})".format(i_fold + 1, n_fold, i_seed + 1, n_seed))

        # dataset
        X_train, Y_train = X.iloc[train_idx], Y[train_idx]
        X_val, Y_val = X.iloc[val_idx], Y[val_idx]


        # train
        running_train(X_train, Y_train, X_val, Y_val, i_fold=i_fold, seed=seed)

        # predict on oof
#         print('predict on oof...', end='')
        model = CatBoostClassifier(thread_count=-1)
        model.load_model( OUTPUT_PATH / 'models' / f'catboost_{i_fold}_{seed}_{CURRENT_TIME}')

        prediction = model.predict_proba(X_val)[:, 1]
        #prediction = model.predict(X_val)
        
        oof[val_idx, i_seed] = prediction
#         print('  done.')

Seed: 948, 1/5
Seed: 534, 2/5
Seed: 432, 3/5
Seed: 597, 4/5
Seed: 103, 5/5


In [ ]:
prediction_threshold = 0.16
funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
if n_fold != 1:
    Y_predicted = (np.mean(oof, axis = 1) > prediction_threshold).astype(int)
    Y_test = funnel[['client_id', 'sale_flg']].set_index('client_id')
    test_funnel =  funnel.set_index('client_id')
if n_fold == 1 and n_seed == 1:
    Y_predicted = (prediction > prediction_threshold).astype(int)
    Y_test = funnel[['client_id', 'sale_flg']].iloc[split_indexes[0][1]].set_index('client_id')
    test_funnel = funnel.iloc[split_indexes[0][1]].set_index('client_id')

In [ ]:
try: 
    os.mkdir(OUTPUT_PATH / 'scoring')
except:
    shutil.rmtree(OUTPUT_PATH / 'scoring')
    os.mkdir(OUTPUT_PATH / 'scoring')

In [ ]:
public_score, private_score = local_scorer.get_score(test_funnel, Y_predicted, Y_test)

In [ ]:
validation_precision = precision_score(Y_test['sale_flg'], Y_predicted)
validation_recall = recall_score(Y_test['sale_flg'], Y_predicted)
validation_f1 = f1_score(Y_test['sale_flg'], Y_predicted)
print(f'Public ANIC {public_score} Private ANIC {private_score}')
print(f'ANIC {1/3*public_score+ 2/3 * private_score}')
print(f'Precision {validation_precision} | Recall {validation_recall} | F1 {validation_f1}')

In [19]:
validation_precision = precision_score(Y_test['sale_flg'], Y_predicted)
validation_recall = recall_score(Y_test['sale_flg'], Y_predicted)
validation_f1 = f1_score(Y_test['sale_flg'], Y_predicted)
print(f'Public ANIC {public_score} Private ANIC {private_score}')
print(f'ANIC {1/3*public_score+ 2/3 * private_score}')
print(f'Precision {validation_precision} | Recall {validation_recall} | F1 {validation_f1}')

Public ANIC 5443.947418364507 Private ANIC 5826.64704507396
ANIC 5699.080502837475
Precision 0.5039475644272308 | Recall 0.961352657004831 | F1 0.6612587959343237


In [21]:
oof.shape

(21498, 5)

In [24]:
oof_cb = np.mean(oof, axis=1)
oof_cb.shape

(21498,)

In [23]:
oof_lgb = np.load('oof_lgb.npy')
oof_lgb.shape

(21498,)

In [25]:
np.save('oof_cb.npy', oof_cb)

In [26]:
Y_test.shape

(21498, 1)

In [44]:
np.linspace(0, 1, 101)[10:40]

array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39])

In [45]:
# prediction_threshold = 0.16
w_range = np.linspace(0, 1, 11)
t_range = np.linspace(0, 1, 101)[10:40]
results = []
for w in w_range:
    for t in tqdm(t_range):
        Y_predicted = (w * oof_cb + (1 - w) * oof_lgb > t).astype(int)
        public_score, private_score = local_scorer.get_score(test_funnel, Y_predicted, Y_test)
        anic = public_score / 3 + private_score * 2 / 3
        results.append((w, t, anic))
    # validation_precision = precision_score(Y_test['sale_flg'], Y_predicted)
    # validation_recall = recall_score(Y_test['sale_flg'], Y_predicted)
    # validation_f1 = f1_score(Y_test['sale_flg'], Y_predicted)
    # print(f'Public ANIC {public_score} Private ANIC {private_score}')
    # print(f'ANIC {1/3*public_score+ 2/3 * private_score}')
    # print(f'Precision {validation_precision} | Recall {validation_recall} | F1 {validation_f1}')

100%|██████████| 30/30 [00:46<00:00,  1.54s/it]


In [49]:
sorted(results, key=lambda x: x[2], reverse=True)[0]

(0.6000000000000001, 0.25, 5713.325276769933)

In [162]:
retrain_after_valid = False
if retrain_after_valid:
    running_train(X, Y, None, None, i_fold=-1, seed=4444)

In [24]:
if USE_WANDB:
    wandb.run.summary["validation_f1"] = validation_f1
    wandb.run.summary["anic"] = 1/3*public_score+ 2/3 * private_score

In [27]:
make_submission = True
if make_submission:
    public_anic = float(input())
    wandb.run.summary["public_anic"] = public_anic

 5800.68066785


In [28]:
if USE_WANDB:
    run.finish()

validation_f1,0.66126
anic,5699.0805
public_anic,5800.68067
